In [1]:
import os

In [2]:
%pwd

'c:\\Users\\admin\\Desktop\\JI\\Project\\capstone\\milk_adulteration_detection\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\admin\\Desktop\\JI\\Project\\capstone\\milk_adulteration_detection'

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    


In [14]:
from detection.constants import *
from detection.utils.common import read_yaml, create_directories

In [18]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path)
        )

        return prepare_base_model_config

In [10]:
import os
import urllib.request as request
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import find_peaks
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import xgboost as xgb

In [21]:
import joblib

In [36]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config


    
    
    
    def get_base_model(self):

        self.model = xgb.XGBClassifier()

        self.save_model(self, path=self.config.base_model_path, model=self.model)


    
    @staticmethod
    def _prepare_full_model(model):
        
        full_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
        )

        self.save_model(self, path=self.config.updated_base_model_path, model=self.full_model)

    
    @staticmethod
    def save_model(self, path: Path, model: xgb):
        joblib.dump( model, path)

In [37]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-06-08 23:52:29,609: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-08 23:52:29,610: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-08 23:52:29,610: INFO: common: created directory at: artifacts]
[2024-06-08 23:52:29,619: INFO: common: created directory at: artifacts/prepare_base_model]
